# LiDAR Development Testing Notebook

This notebook serves as a testing tool for dealing with the LiDAR sensor object in tesse. Without Unity unit tests, it is hard to do any kind of testing that isn't end-to-end.

In [ ]:
from tesse.env import Env
from tesse.msgs import *

import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
import defusedxml.ElementTree as ET
import numpy as np

In [ ]:
env = Env()


### Get LiDAR Params

In [ ]:
response = env.request(LidarInformationRequest())

if response:
    print(response)
    print(response.metadata)

### Set LiDAR Params


In [ ]:
response = env.request(SetLidarParametersRequest(scanner=Lidar.ALL,
                                                 min_angle=-3.14,
                                                 max_angle=3.14,
                                                 max_range=50,
                                                 ray_count=1000))

if response:
    print(response)
    print(response.metadata)
    
_ = env.request(SetLidarParametersRequest(scanner=Lidar.UNDER,
                                          min_angle=3.14,
                                          max_angle=6.28,
                                          max_range=50,
                                          ray_count=1000))

### Set LiDAR Position and Orientation

In [ ]:
response = env.request(SetLidarPositionRequest(Lidar.ALL,-0.02,1.02,1.7))

if response:
    print(response)
    print(response.metadata)
    
response = env.request(SetLidarOrientationRequest(Lidar.ALL,0,0,0.0001,1))

if response:
    print(response)
    print(response.metadata)

### Get LiDAR Data and Plot

In [ ]:
def plot_lidar(ax, ranges, min_angle, max_angle):
    increment = (np.abs(min_angle) + np.abs(max_angle)) / len(ranges)
    angles = np.array([min_angle + i*increment for i in range(len(ranges))])
    
    ax.scatter(angles, ranges, s=1)

min_angle = 0
max_angle = np.pi
_ = env.request(SetLidarParametersRequest(scanner=Lidar.ALL,
                                                 min_angle=min_angle,
                                                 max_angle=max_angle,
                                                 max_range=50,
                                                 ray_count=1000))

plt.ion()
plt.show()

while True:
    response = env.request(LidarDataRequest())
    if response:
        clear_output(wait=True)
        ax = plt.subplot(111,polar=True)
        plot_lidar(ax, response.scans[0], min_angle, max_angle)
        plot_lidar(ax, response.scans[1], -np.pi, 0)
        plot_lidar(ax, response.scans[2], min_angle, max_angle)
    else:
        print('no response')
    
    plt.show()